필요한 API 불러오기

In [1]:
import sys, os
sys.path.append(os.pardir)
import pandas as pd
import numpy as np
import keras
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.image as img
import matplotlib.pyplot as plt
from augmentation.imageAugmentation import DataAugSaveCsv

검증 데이터 추출

In [2]:
def get_validation_data(x_data, y_data, valid_per):
    valid_percent = round(len(x_data) * valid_per)
    x_valid = x_data[:valid_percent]
    y_valid = y_data[:valid_percent]
    x_data = x_data[valid_percent:]
    y_data = y_data[valid_percent:]
    return (x_data, y_data, x_valid, y_valid)

이미지 데이터: x, 라벨 데이터: y 추출

In [3]:
def get_x_batch(df):
    x_batch = []
    data_count = df.shape[0]
    per = data_count // 10
    percent = 0
    for data_index in range(data_count):
        image = df.iloc[data_index, 1:]
        x_batch.append(image.tolist())
        if(data_index % per == 0):
            print(percent, "%", end = '->')
            percent += 10
    print()
    x_batch = np.asarray(x_batch)
    x_batch = x_batch.reshape(x_batch.shape[0], 32, 32, 3).astype("float32")
    return x_batch

def get_y_batch(df):
    y_batch = df.iloc[:,0]
    y_batch = np_utils.to_categorical(y_batch)
    return y_batch

def get_deeplearning_data(df):
    x_data = get_x_batch(df)
    y_data = get_y_batch(df)
    del(df)
    return (x_data, y_data)

이미지 데이터

In [5]:
train_file_name_list = ['ㄱ1.png','ㄴ1.png','ㄷ1.png','ㄹ1.png',
                 'ㄱ2.png','ㄴ2.png','ㄷ2.png','ㄹ2.png',
                 'ㄱ3.png','ㄴ3.png','ㄷ3.png','ㄹ3.png',
                 'ㄱ4.png','ㄴ4.png','ㄷ4.png','ㄹ4.png',
                 'ㄱ5.png','ㄴ5.png','ㄷ5.png','ㄹ5.png',
                 'ㄱ6.png','ㄴ6.png','ㄷ6.png','ㄹ6.png',
                 'ㄱ7.png','ㄴ7.png','ㄷ7.png','ㄹ7.png',
                 'ㄱ8.png','ㄴ8.png','ㄷ8.png','ㄹ8.png',
                 'ㄱ9.png','ㄴ9.png','ㄷ9.png','ㄹ9.png',
                 'ㄱ10.png','ㄴ10.png','ㄷ10.png','ㄹ10.png',
                 'ㄱ11.png','ㄴ11.png','ㄷ11.png','ㄹ11.png']
test_file_name_list = ['ㄱt1.png','ㄴt1.png','ㄷt1.png','ㄹt1.png',
                      'ㄱt2.png','ㄴt2.png','ㄷt2.png','ㄹt2.png',
                      'ㄱt3.png','ㄴt3.png','ㄷt3.png','ㄹt3.png']
file_path = 'E:/data/한글 data/가나다라data/'

이미지 데이터 증식, csv 저장

In [6]:
DataAugSaveCsv(train_file_name_list, file_path,'train-korean-4','E:/data/한글 data/', 1000)
DataAugSaveCsv(test_file_name_list, file_path,'test-korean-4','E:/data/한글 data/', 1000)

0 %->10 %->20 %->30 %->40 %->50 %->60 %->70 %->80 %->90 %->100 %->
done!
0 %->10 %->20 %->30 %->40 %->50 %->60 %->70 %->80 %->90 %->100 %->110 %->
done!


증식된 이미지 데이터 불러오기

In [4]:
train_df = pd.read_csv("E:/data/한글 data/train-korean-4.csv")
x_train, y_train = get_deeplearning_data(train_df)
test_df = pd.read_csv("E:/data/한글 data/test-korean-4.csv")
x_test, y_test = get_deeplearning_data(test_df)

0 %->10 %->20 %->30 %->40 %->50 %->60 %->70 %->80 %->90 %->
0 %->10 %->20 %->30 %->40 %->50 %->60 %->70 %->80 %->90 %->


검증 데이터 추출

In [5]:
x_train, y_train, x_valid, y_valid = get_validation_data(x_train, y_train, 0.2)

모델 정의

In [7]:
print("훈련 데이터 개수:", len(x_train))
print("테스트 데이터 개수:", len(x_test))
print("검증 데이터 개수:", len(x_valid))

훈련 데이터 개수: 35200
테스트 데이터 개수: 12000
검증 데이터 개수: 8800


In [ ]:
def define_model():
    model = keras.models.Sequential([
        keras.layers.Conv2D(32, 3, kernel_initializer = "he_uniform", activation = "relu", padding = "same",
                            input_shape = [32, 32, 3]),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(32, 3, kernel_initializer = "he_uniform", activation = "relu", padding = "same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.2),
        keras.layers.Conv2D(64, 3, kernel_initializer = "he_uniform", activation = "relu", padding = "same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, 3, kernel_initializer = "he_uniform", activation = "relu", padding = "same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.3),
        keras.layers.Conv2D(128, 3, kernel_initializer = "he_uniform", activation = "relu", padding = "same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, 3, kernel_initializer = "he_uniform", activation = "relu", padding = "same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.4),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation = "relu", kernel_initializer = "he_uniform"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4, activation = "softmax")
    ])
    return model

정의된 모델 불러오고 compile 하기

In [8]:
korean_model = define_model()

In [9]:
korean_model.compile(loss = "categorical_crossentropy", optimizer = "sgd", metrics = ["accuracy"])

학습된 모델을 저장할 폴더 생성

In [ ]:
def create_model_dir(model_dir):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

In [11]:
model_dir = './한글 classifier model/'
create_model_dir(model_dir)

modelpath = model_dir + '{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = modelpath,
                               monitor = "val_loss",
                               verbose = 1,
                               save_best_only = True)
early_stopping_callback = EarlyStopping(monitor = "val_loss", patience = 5)

In [12]:
history = korean_model.fit(x_train,
                     y_train,
                     batch_size = 64,
                     validation_data = (x_valid, y_valid),
                     epochs = 20,
                     verbose = 0,
                     callbacks = [early_stopping_callback, checkpointer])

print("\n Test Accuracy: %.4f" % korean_model.evaluate(x_test, y_test)[1])

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()